# Pandas Introduction

## Reading files into DataFrames

In Pandas, data is accessed through a [DataFrame](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe). A DataFrame is a 2D data structure where each column may contain different data types, from numeric series to complex structures. In most cases, you can think on DataFrames as _tables_.

The [IO API](https://pandas.pydata.org/pandas-docs/stable/io.html) has different methods to read different formats, most common one is text-delimited files:

In [ ]:
import pandas as pd
data = pd.read_csv("./data/goog.csv")

From this point we can access the data using different methods. For instance, `head(n)` will retrieve the first `n` columns:

In [ ]:
data.head(2)

DataFrame also allows _range_ expressions, by using `[]` or `loc`.

In [ ]:
data[5:7]

In [ ]:
data.loc[5:7]

In [ ]:
edge_case = pd.DataFrame({'val': ['a', 'b', 'c', 'd']}, index=[10, 20, 30, 40])
print(edge_case.loc[20:30])

Single row access is done using `iloc` attribute:

In [ ]:
data.iloc[5]

In [ ]:
data.iloc[5].Open

All methods and attributes a DataFrame supports can be checked in the [API documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html). Note that althought Pandas support Python's index operator `[]`, the recommended way to access values or ranges is to use `loc` for labels and `iloc` for positions. There's more information about it in the [Indexing and selecting data](https://pandas.pydata.org/docs/user_guide/indexing.html) chapter of Pandas documentation.

### Data formats and compression

Pandas can read and write multiple data formats, using `read_X` and `to_X` methods. Parameters are described in the [IO API documentation](https://pandas.pydata.org/pandas-docs/stable/io.html).

One of the popular formats is [PyArrow](https://arrow.apache.org/docs/python/pandas.html), that has relatively good integration with Pandas. Normally, you will be interested to use a binary column-oriented format called [Feather](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#feather), created from the Arrow project.

In [ ]:
data = pd.read_feather('./data/goog.feather')
data.dtypes

Notice that in old versions of PyArrow, the index is lost when saving the data. You may want to create it again by doing:

In [ ]:
data_indexed = data.set_index('Date')
data_indexed.head(1)

You can also replace the index in-place, without creating a copy:

In [ ]:
data.set_index('Date', inplace=True)
data.head(1)

Similarly, you can write Feather files using `to_feather` function. Remember to drop the index first:

In [ ]:
data = pd.read_csv("./data/goog.csv", index_col=0)
data.reset_index().to_feather('data/goog.feather')

Another widely use column-oriented format is [Parquet](https://parquet.apache.org/), a native Apache Hadoop ecosystem data storage format.

In [ ]:
data = pd.read_parquet("./data/goog.parquet")

Notice the size difference even with a small dataset

In [ ]:
!ls -lh ./data/goog.csv ./data/goog.parquet

Pandas also accepts compressed files, for instance in `gzip` format:

In [ ]:
data = pd.read_csv('./data/goog.csv.gz')
data.head(1)

Remember that you don't need anything fancy to compress files, just use shell tools:

```
$ gzip -9 data_file.csv
```

## Accessing Data

### Column Indexing

By default, `read_csv` will not assume any index in the data. In our case we have a _natural_ index in the first column, so we can read the file again to make sure it's incorporated in the DataFrame:

In [ ]:
data = pd.read_csv("./data/goog.csv", index_col=0)
data.head(2)

And then we can use range expressions to fetch _years_:

In [ ]:
data["2016":"2015"].head(2)

Or _month_ ranges:

In [ ]:
data["2016-05":"2015-04"].head(2)

Notice that the index is set as an string instead of a timestamp. This could produce weird results when getting date ranges from the DataFrame. If you want to parse the dates, you have to set the flag `parse_dates=True` when reading the file.

In [ ]:
data = pd.read_csv("./data/goog.csv", index_col=0, parse_dates=True)
data.index[0]

Then, for accessing a particular range remember the to use the `.loc` function instead of the array accessor:

In [ ]:
data.loc['2016'].head(2)

You can also use a date or a [Period](https://pandas.pydata.org/docs/reference/api/pandas.Period.html) object. Both are also sensible to the DataFrame ordering:

In [ ]:
from datetime import datetime
data[datetime(2015,1,1):datetime(2016,1,1)]

In [ ]:
data.sort_index(ascending=True, inplace=True)
dfN = data[datetime(2015,1,1):datetime(2016,1,1)]
dfN.iloc[0:1].index, dfN.iloc[-1:].index

In [ ]:
dfQ = data.loc[str(pd.Period("2015"))]
dfQ.iloc[0:1].index, dfQ.iloc[-1:].index

There's also a method called `.query` that would allow us to perform queries over the data:

In [ ]:
data.query('Close > 600 & Volume > 33000000')

### Adding or removing columns

In [ ]:
data["Diff"] = data.Close - data.Open
data[["Open", "Close", "Diff"]].head(2)

In [ ]:
del data["Diff"]
data.columns

### Renaming a Column

In [ ]:
data.rename(columns={'Adj Close': 'adj_close'})

### Parsing Dates

You would normally use `parse_dates=True` or `parse_dates=[0]` to automatically process dates when reading files. In some cases you would like to perform that manually by using `pd.to_datetime` function:

In [ ]:
df2 = pd.read_csv("./data/goog.csv")
df2['Month'] = pd.to_datetime(df2['Date']).dt.month
df2.head(2)

### Grouping

In [ ]:
df2.groupby(df2['Month'])['Close'].mean()

### Multi-Indexes

Pandas is able to treat data that has indexes in multiple levels. For instance, the `prices.csv` data set has two initial columns with the stock symbol and the date of the prices. That's described in [Hierarchical indexing](https://pandas.pydata.org/pandas-docs/stable/advanced.html#advanced-hierarchical) documentation.

In [ ]:
pd.read_csv('./data/prices.csv').head(2)

We can easily tell Pandas that first two columns are the index by using the `index_col` parameter. Let's create a data frame named `prices`, containing price data for several stocks in a time range.

In [ ]:
prices = pd.read_csv('./data/prices.csv', index_col=[0,1])
prices.head(5)

In [ ]:
prices.index

To query for a particular index, you can use `get_level_values` function and `isin`:

In [ ]:
msft = prices[prices.index.get_level_values('Symbol').isin(['msft'])]
msft.head()

Also, `query` method is also available for multi-indexes:

In [ ]:
prices.query('Symbol == "msft"').head()

Once filtering is done, you might want to drop `Symbol` index:

In [ ]:
msft.index = msft.index.droplevel(0)
msft.head()

In [ ]:
aapl = prices[prices.index.get_level_values('Symbol').isin(['aapl'])]
aapl.index = aapl.index.droplevel(0)
aapl.head()

All that operations can be simplified by using `loc` function

In [ ]:
prices.loc['aapl'].head()

### Stacking

Using `stack` and `unstack` you can easily convert multi-indexed data into different columns. Notice that in the following example, we're using column `1` as the first one in the index.

In [ ]:
prices = pd.read_csv('./data/prices.csv', index_col=[1,0], usecols=[0,1,3], parse_dates=[1])
prices.head(5)

In [ ]:
prices.unstack().head(2)

We recommend to read the Pandas documentation about [pivoting data](https://pandas.pydata.org/docs/user_guide/reshaping.html#pivot-tables).

## Series

All columns in the previous DataFrame are defined as [_Series_](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#series). A Serie is a data bag indexed by date, in most cases.

In [ ]:
data.Close[:2]

In [ ]:
type(data)

In [ ]:
type(data.Close)

[Series API](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html#pandas.Series) has multiple methods, but one of the most interesting ones is the `describe` one:

In [ ]:
data.Close.describe()

### Sampling

In [ ]:
data.Close.sample(n=3)

There are two interesting parameters to separate training data from model testing data:
- Use `frac` to get the % of items from the whole set to retrieve
- If you want sets to be consistent between runs, use `random_state`.

In [ ]:
data.Close.sample(frac=0.003, random_state=1)

There are also two functions to retrieve the largest and smallest values of a set:

In [ ]:
data.Close.nlargest(3)

### Querying

In [ ]:
data[data.Close > 1220]

In [ ]:
data.query("Close > 1220")

### Masking 

There's also two functions, `mask` and `where`, that you can use to discard or include values. In this case it's being is used to filter out values greater than 685, and fill the values with -42. If fill value is omited, `NaN` is used instead.

In [ ]:
data.Close.where(lambda x: x > 685, -42).head(5)

 Notice that `mask` and `where` do the opposite:

In [ ]:
data.Close.mask(lambda x: x > 685, -42).head(5)

### Pipes

It is possible to apply a function to every value of the Series:

In [ ]:
def double(x):
    return x * 2

data.Close.pipe(double).pipe(lambda x: x + 5).head(2)

The `pipe` operation can be also applied to DataFrames:

In [ ]:
def double_close(df):
    df['Double Close'] = df['Close'] * 2
    return df

data.pipe(double_close).head(2)

### Apply

The `apply` method allows to execute arbitrary functions over a Series.

In [ ]:
data.Close['2016'].apply(lambda x: x * 2)

It can also be applied to a DataFrame directly, by selecting the axis - normally `0` for the index and `1` for the DataFrame columns.

In [ ]:
import numpy as np
data.apply(np.mean, axis=0)

In [ ]:
data.apply(np.mean, axis=1)

### Reading directly into a series

Instead of reading files as `DataFrames`, it's possible to read a `Series` directly, using column 0 as date, and colum 4 as the `Series` value. Do not forget to apply the `.squeze` function afterwards.

In [ ]:
series = pd.read_csv("./data/goog.csv", header=0, index_col=0, usecols=[0, 4], parse_dates=True).squeeze()
series.head()

---

### Exercise 1.1

Read file `prices.csv` and filter symbol `tsla`. Select 90% of `Close` column for training and reserve 10% for model later checking. Try to randomise data picked.

You may want to check [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/).

### Exercise 1.2

From `prices.csv` file, read `msft` data and write it as a Feather file in `data/msft.feather`.

### Exercise 1.3

Add a boolean column to previous `msft` data frame indicating if trades open with values greater than 100.

### Exercise 1.4

Look in Pandas documentation for __styling functions__ and mark in red the `close` values below 30.

---

## Time-Series

Our series can be [resampled](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html) to months, using mean close price for each one.

- M: Months
- Q: Quarters
- A: Years

The `resample` function just returns an object represeting the resampling. We have to pick one of the many functions to extract the value for each new of the rows generated (months, quarters or year rows). The available offset strings can be found [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects).

In [ ]:
series = pd.read_csv("./data/goog.csv", header=0, index_col=0, usecols=[0, 4], parse_dates=True).squeeze()
series.resample("A")

### Padding

First thing we can do is to get the latest value known in each range, `ffill` method. Don't use `pad` or `fillna(method="ffill")` functions, since they are deprecated.

In [ ]:
series['2004-12-31']

For brevity, there's a function called `ffill` that calls to `fillna` internally:

In [ ]:
series.resample("").ffill().head()

### Computing range mean, min or max values

And we can also get the mean values for each range. Instead of `fillna`, use `mean` function:

In [ ]:
series["2004"].mean()

In [ ]:
series.resample("A").mean().head()

In [ ]:
series.resample("A").max().head()

Or even `sum` all the values in a range - although in this case it doesn't make too much sense.

In [ ]:
series.resample("A").sum().head()

### Ploting resampled series for better understanding

In [ ]:
series.head()

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
series.plot(figsize=(10,5))
series.resample("M").mean().plot(linewidth=3)
plt.show()

### OHLC: Open, High, Low, Close

A very interesting function is `ohlc`, that computes several values for a date range.

In [ ]:
series.resample("A").ohlc()

### Custom ranges

Every two years:

In [ ]:
series.resample("2AS").mean().head()

There is more information for those magic strings like `2AS in [the documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects).

### Upsampling

Apart of grouping data (or _downsampling_) we can also calculate interpolated values. For instence, we're going to calculate values for each quarter, using `"Q"` or `"4M"` parameters.

First we need to get the data downsampled to year values, using latest one.

In [ ]:
msft_year = series.resample("A").mean()
msft_year

Then we can interpolate some values. Normally you would use `ffill`, optionally setting the maximum number of rows to fill before stopping propagation and filling with `NaN`.

In [ ]:
rs = msft_year.resample("Q")
max_propagated = 2
pd.DataFrame({ 'mean': rs.mean(), 'ffill': rs.ffill(), 
              'ffill_max': rs.ffill(max_propagated) })[:9]

---

### Exercise 1.5

Read CSV data for Tesla `tsla` directly into a `Series` and compute _closing_ values for each quarter.

---

## Cookbook


### Combining Series into new DataFrame

In [ ]:
msft_aapl = pd.DataFrame({ 'msft': msft.Close, 'aapl': aapl.Close })
msft_aapl[:3]

By default it uses closed ranges.

### Column projections

In [ ]:
from sample_data import stock_prices
aapl = stock_prices('aapl')

In [ ]:
aapl

In [ ]:
aapl.loc[:,"Year"] = pd.to_datetime(aapl.index).map(lambda d: d.year)
aapl[:2]

### Using custom indexes

In [ ]:
dates = pd.date_range('2015-10', '2016-02', freq='M')
dates

In [ ]:
from pandas import Series
Series(series, index=dates)

### Install packages in Colab Notebooks

Very simple, use `pip` or `conda`, by appending a `!` in a cell for executing in a shell

`!pip install package_name`.